# Context

## 分析合成控制结果

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
from matplotlib import pyplot as plt

from src import NATURE_PALETTE as nature_colors
from mksci_font import config_font

config_font()
THESIS = r"../PhD_Thesis/img/ch5/"

In [ ]:
from src.multi_synth import MultiSynth
from core.overall_analysis import add_statistic_items
import os
from hydra import compose, initialize


with initialize(version_base=None, config_path="../config"):
    os.chdir("../")
    cfg = compose(config_name="synth")

exp87 = MultiSynth.load_from_pickle("model/model_87.pkl")
exp98 = MultiSynth.load_from_pickle("model/model_98.pkl")
statistic_87 = add_statistic_items(exp87, cfg)
statistic_98 = add_statistic_items(exp98, cfg)

In [ ]:
from core.overall_analysis import analysis_exp

msg = analysis_exp(exp87)
print(msg)

In [ ]:
msg = analysis_exp(exp98)
print(msg)

In [ ]:
from core.plots import plot_comprehensive_fig1

fig1 = plot_comprehensive_fig1(exp87, exp98, cfg, save="figs/outputs/main_results2.pdf")

In [ ]:
from mksci_font import update_font, show

mapping = {
    "Observation": "观测用水量",
    "Prediction": "模拟用水量",
    "Drying-up": "黄河断流事件",
    "10yrs-Avg. drought index": "10年平均干旱指数",
    "Drought": "干旱指数",
    "Drought index": "干旱指数",
    "Study period division": "研究时段",
    "87-WAS": "“八七”分水方案",
    "98-UBR": "流域统一调度",
    "Drying-up / days": "断流时长/天",
    "Total water use": "流域总用水量",
    "Before": "政策干预前",
    "After": "政策干预后",
    "IS1: 87-WAS": "“八七”分水方案",
    "IS2: 98-UBR": "流域统一调度",
}


def update_fonts_and_save(fig, name, mapping):
    updated_fig = update_font(fig, mapping)
    path = os.path.join(THESIS, name)
    updated_fig.savefig(path, dpi=300)
    return updated_fig


update_fonts_and_save(fig1, "ch5_results.png", mapping)

In [ ]:
mapping = {
    "Root Mean Square Error (RMSE)": "均方根误差 (RMSE)",
    "Normalized Total water use": "标准化后的区域用水量",
    "Qinghai": "青海",
    "Gansu": "甘肃",
    "Ningxia": "宁夏",
    "Neimeng": "内蒙古",
    "Shanxi": "陕西",
    "Shaanxi": "山西",
    "Henan": "河南",
    "Shandong": "山东",
}

fig87 = exp87.plot_panels("trend", figsize=(12, 6), save="figs/outputs/87placebo.pdf")
update_fonts_and_save(fig87, "trend_87.png", mapping)


fig98 = exp98.plot_panels("trend", figsize=(12, 6), save="figs/outputs/98placebo.pdf")
update_fonts_and_save(fig98, "trend_98.png", mapping)

In [ ]:
fig87_rmse = exp87.plot_panels(save="figs/outputs/87rmse.pdf")
update_fonts_and_save(fig87_rmse, "rmse_87.png", mapping)

fig98_rmse = exp98.plot_panels(save="figs/outputs/98rmse.pdf")
update_fonts_and_save(fig98_rmse, "rmse_98.png", mapping)


对DSC的有效性测试有两种方法：
- 比较后期和前期的重建；
- 通过安慰剂分析测试鲁棒性。

\begin{equation}
    \text{RMSE} = \sqrt{\frac{1}{n}\sum_{i=1}^{n}{(y_i-\hat{y}_i)}^2} 
\end{equation}

where $n$ is the number of observations, $y_i$ are the actual values, and $\hat{y}_i$ are the predicted values.

In [ ]:
exp87.rmse_report()

In [ ]:
exp98.rmse_report()

使用合成控制法的安慰剂检验，使用均值+-两倍[标准差](https://www.zhihu.com/question/22864111)对显著性进行判断。

## Compare 87-98 regulating

**Abstract:**
One-sentence description

**Description:**
In the following cell, I...


In [ ]:
from src.plots import correlation_analysis

correlation_analysis(statistic_98, xs=["YR_WU", "satisfied"], y="diff_ratio")

In [ ]:
from core.overall_analysis import get_compare_df

rename = {
    "Qinghai": "青海",
    "Gansu": "甘肃",
    "Ningxia": "宁夏",
    "Neimeng": "内蒙古",
    "Shanxi": "陕西",
    "Shaanxi": "山西",
    "Henan": "河南",
    "Shandong": "山东",
}

compare_df = get_compare_df(statistic_87, statistic_98)
compare_df_ch = compare_df.rename(rename, axis=0)

In [ ]:
from core.plots import comparison_plot

mapping = {
    "Root Mean Square Error (RMSE)": "均方根误差 (RMSE)",
    "Normalized Total water use": "标准化后的区域用水量",
    "Major water users": "主要用水区域",
    "Extra WU over the estimation": "用水超预期比例",
    "WU ratio": "用水量占比",
    "87-WAS": "“八七”分水",
    "98-UBR": "统一调度",
    "Quota": "分水配额",
    "WU ratio": "用水量占比",
}


fig2 = comparison_plot(compare_df, save="figs/outputs/fig3.pdf")
fig2_ch = comparison_plot(compare_df_ch)
update_fonts_and_save(fig2_ch, "fig3.png", mapping)

In [ ]:
compare_df["87_ratio"].iloc[:4].mean()
compare_df["98_ratio"]

In [ ]:
compare_df["98_ratio"][compare_df["98_ratio"] < 0].mean()

In [ ]:
compare_df.loc["Neimeng", :]

In [ ]:
(
    compare_df.loc["Neimeng", "98_ratio"] - compare_df.loc["Neimeng", "87_ratio"]
) / compare_df.loc["Neimeng", "87_ratio"]